<a href="#my_image_id">See Image</a>

## Kinematics

The kinematics of a robot used in the thesis are introduced in this section along with some basic concepts and algorithmic implementations. The first subsection discusses the conversions between coordinate frames and the selected convention. Next, a brief explanation of the Geometric Jacobian computation procedure and forward kinematics is given.

### Denavit-Hartenberg Convention and Transformation Matrix

Although reference frames attached to the links can be chosen arbitrarily, it is practical to select them systematically. A widely used convention for describing the 
kinematic geometry of robotic arms is the Denavit-Hartenberg (DH) convention <cite id="gx4bn"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite><cite id="10z4u"><a href="#zotero%7C17740253%2F3J5DS9LQ">(Denavit &#38; Hartenberg, 2021)</a></cite> and is followed in this thesis.

The homogeneous transformation between frame $i$ and $i-1$ is obtained by multiplying the rotation $\mathbf{T}_{R_z}$ around the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_z}$ along the $\hat{\mathbf{z}}_{i-1}$-axis, the translation $\mathbf{T}_{T_x}$ along the $\hat{\mathbf{x}}_i$-axis, and the rotation $\mathbf{T}_{R_x}$ around the $\hat{\mathbf{x}}_i$-axis, as follows:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = \mathbf{T}_{R_z}(\theta_i) \mathbf{T}_{T_z}(d_i) \mathbf{T}_{T_x}(a_i) \mathbf{T}_{R_x}(\alpha_i)
\end{equation}

Combining these, the homogeneous transformation matrix is:

\begin{equation}
{}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i) = 
\begin{bmatrix}
c_{\theta_i} & -s_{\theta_i} c_{\alpha_i} & s_{\theta_i} s_{\alpha_i} & a_i c_{\theta_i} \\
s_{\theta_i} & c_{\theta_i} c_{\alpha_i} & -c_{\theta_i} s_{\alpha_i} & a_i s_{\theta_i} \\
0 & s_{\alpha_i} & c_{\alpha_i} & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
\end{equation}

In this expression, $c_{\theta_i}$ and $s_{\theta_i}$ represent $\cos(\theta_i)$ and $\sin(\theta_i)$, respectively. Notably, the homogeneous transformation ${}^{i-1}\mathbf{T}_i(\theta_i, d_i, a_i, \alpha_i)$ depends on a single variable once the reference frame is established and the other parameters are fixed. This variable is determined by the type of joint: $d_i$ for prismatic joints and $\theta_i$ for revolute joints. A more elaborate theory can be found in <cite id="zhen5"><a href="#zotero%7C17740253%2FLY2FP5ZV">(Ghosal, 2006)</a></cite>.

In [1]:
def dh_transform(a, alpha, d, theta):
    """
    Compute Denavit-Hartenberg transformation matrix.
    """
    return np.array([
        [np.cos(theta), -np.sin(theta) * np.cos(alpha),  np.sin(theta) * np.sin(alpha), a * np.cos(theta)],
        [np.sin(theta),  np.cos(theta) * np.cos(alpha), -np.cos(theta) * np.sin(alpha), a * np.sin(theta)],
        [0,             np.sin(alpha),                 np.cos(alpha),                 d],
        [0,             0,                             0,                             1]
    ])

### Forward Kinematics

In a robotic manipulator, the links are generally rigid bodies that form the robot’s structure, while the joints are the movable components that join two or multiple links. Joints can be  with an actuator (active joints), or without an actuator (passive joints).Joints commonly found in serial robots are usually prismatic or revolute. Each type of such joint adds one degree of freedom (dof) to the system: the joint position $d_i$ for prismatic joints or the joint angle $q_i$ for revolute joints. The entire kinematic structure is typically referred to as a chain, which can be of two types:

* Open chain: In this configuration, the two extremes, also known as the base and the end-effector are connected by a single path in the structure.
* Closed chain: The kinematic structure of this configuration has loops.

The robot used in this thesis consists only of active revolute joint in an open chain configuration.

A common problem in robotics is determining the Cartesian position and orientation $\mathbf{x} \in \mathbb{R}^6$ of a specific point on a link given the joint angles $\mathbf{q} \in \mathbb{R}^n$. This problem is known as forward kinematics, which can be efficiently solved using homogeneous transformations. The position of the origin of the end-effector’s reference frame $\text{RF}_e$ in the base frame $\text{RF}_b$ can be determined using the following equation:

$$
{}^b\mathbf{T}_e(\mathbf{q}) = {}^b\mathbf{T}_0 \, {}^0\mathbf{T}_1(q_1) \cdots {}^{n-1}\mathbf{T}_e(q_n) \, {}^n\mathbf{T}_e,
$$

where the two constant homogeneous transformations required to convert a point from frame 0 to the base frame and from the end-effector frame to frame $n$, respectively, are denoted by ${}^b\mathbf{T}_0$ and ${}^n\mathbf{T}_e$.

 The following algorithmic implementation computes the transformation between all reference frames in the robot defined in the dh parameters.

In [3]:
def compute_forward_kinematics(joint_angles, dh_params):
    """
    Compute the forward kinematics for a robot given DH parameters and joint angles.

    :param dh_params: List of DH parameters [(a, alpha, d, theta), ...]
    :param joint_angles: List of joint angles [theta1, theta2, ..., thetaN]
    :return: List of transformation matrices from base to each joint and the final transformation matrix T_0_n
    """
    num_joints = len(joint_angles)
    T = np.eye(4)
    transformations = [T.copy()]
    
    # Compute forward kinematics to get transformations
    for i, (a, alpha, d, theta) in enumerate(dh_params):
        theta += joint_angles[i]  # Update theta with joint angle
        T_i = dh_transform(a, alpha, d, theta)
        T = np.dot(T, T_i)
        transformations.append(T.copy())
    
    T_0_n = transformations[-1]  # Final transformation matrix to the TCP (end-effector)
    return transformations, T_0_n

###  Geometric Jacobian

The geometric Jacobian matrix, $J(q) ∈ R^(6×n)$, transforms the joint space velocities $q̇$, into Cartesian space velocities $ẋ$. This relationship is represented by the equation:
\begin{equation}
\dot{\mathbf{x}} = 
\begin{bmatrix}
\dot{\mathbf{p}} \\
\boldsymbol{\omega}
\end{bmatrix} 
= \mathbf{J}(\mathbf{q}) \dot{\mathbf{q}},
\end{equation}
where $\dot p = [ \dot x, \dot y, \dot z ]^T $represents the linear velocities and $\omega = [ \omega_x, \omega_y, \omega_z ]^T$ represents the angular velocities of the robot's end-effector in the base frame. The geometric Jacobian is calculated based on the robot's geometry at a specific instance. An elaborate theortical background can be found in [], an algorithmic implementation is shown below. 

In [4]:
def compute_geometric_jacobian(transformations):
    """
    Compute the geometric Jacobian for a robot given the transformation matrices.

    :param transformations: List of transformation matrices from the base to each joint
    :return: Geometric Jacobian matrix (6 x N) in the base frame
    """
    num_joints = len(transformations) - 1  # Excluding the base frame
    jacobian_base = np.zeros((6, num_joints))
    
    # End-effector position
    T_0_n = transformations[-1]
    o_n = T_0_n[:3, 3]

    # Compute each column of the Jacobian in the base frame
    for i in range(num_joints):
        T_0_i = transformations[i]
        o_i = T_0_i[:3, 3]
        z_i = T_0_i[:3, 2]

        jacobian_base[:3, i] = np.cross(z_i, (o_n - o_i))  # Linear part
        jacobian_base[3:, i] = z_i  # Angular part

    return jacobian_base

In [5]:
def convert_jacobian_to_tcp(jacobian_base, T_0_n):
    """
    Convert a Jacobian from the base frame to the TCP frame.
    
    :param jacobian_base: Jacobian matrix in the base frame (6 x 6)
    :param T_0_n: Transformation matrix from the base to the TCP (4 x 4)
    :return: Jacobian matrix (6 x 6) in the TCP frame
    """
    R_0_n = T_0_n[:3, :3]
    T_base_to_tcp = np.block([
        [R_0_n.T, np.zeros((3, 3))],
        [np.zeros((3, 3)), R_0_n.T]
    ])
    
    jacobian_tcp = np.dot(T_base_to_tcp, jacobian_base)
    return jacobian_tcp